## import library
- http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
- http://learningtensorflow.com/index.html
- http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

# 0. Tensorflow basic

- **tf.constant(), tf.Variable()**: 그래프의 객체
- **tf.Session()/ tf.InteractiveSession()**: 그래프를 시작
- **sess.run(c)**: 텐서 'c'를 계산 -> sess.close() 해야함
- **with 절**: tf.Session()은 with 절과 사용, with 절에서 sess.close() 생략가능
- **c.eval()**: sess.run(c)와 같음

### - **tf.variable_scope()**
- 모델에서 필요한 변수들을 관리하는 클래스(변수 범위 만들기)
- tf.get_variable(): 직접호출 없이 변수를 가져오거나 생성(initializer 사용)
- reuse = False: '현재 variable scope 이름 + 제공된 name'이 없으면 생성
- reuse = True: '현재 variable scope 이름 + 제공된 name'이 있으면 반환
- reuse = AUTO_REUSE: 둘다

# 1. Create RNN Cell

In [2]:
#one-hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)


### Create Cell
- tf.variable_scope('one_cell')이라는 객체 -> scope
- cell 정의: 출력의 크기 = hidden_size
- cell 구동: cell, 입력 data 넘겨주면 -> output, 마지막 state를 리턴
- 연산은 graph로 표현: graph는 session 내에서 실행

In [3]:
with tf.variable_scope('one_cell', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    # cell 정의: 출력의 크기 = hidden_size
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    # cell 구동: cell, 입력 data 넘겨주면 -> output, 마지막 state를 리턴

    sess.run(tf.global_variables_initializer())
    # 연산 실행 전 initialize 수행
    pp.pprint(outputs.eval())
    # output: random한 weignt와 연산되어서 나온 값(dim=2)

2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
array([[[-0.26044506,  0.483253  ]]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

### Input data = [h,e,l,l,o]

In [4]:
with tf.variable_scope('two_sequances', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    # 셀 생성
    
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)  
    pp.pprint(x_data)
    # input data(1,5,4): sequence_length = 5
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    # 셀 구동
    
    sess.run(tf.global_variables_initializer())
    # initialize
    pp.pprint(outputs.eval())
    # output data(1,5,2) 연산

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.5163382 ,  0.74790335],
        [ 0.9169806 ,  0.8156855 ],
        [ 0.5477242 ,  0.46063218],
        [ 0.18679112,  0.21059105],
        [-0.11483069, -0.19650726]]], dtype=float32)


![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

### Multiple sequences (문자열 여러 개)
- batch size = 3

In [5]:
with tf.variable_scope('3_batches', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    # 셀 생성
    
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    print(x_data.shape)  
    pp.pprint(x_data)
    # input data(3,5,4): sequence_length = 5
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    # 셀 구동
    
    sess.run(tf.global_variables_initializer())
    # initialize
    pp.pprint(outputs.eval())
    print(outputs)
    # output data(3,5,2)


(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.5162457 ,  0.09800961],
        [ 0.80118716, -0.24169545],
        [ 0.6995623 ,  0.2410628 ],
        [ 0.43763998, -0.04675321],
        [ 0.23257603,  0.34301198]],

       [[ 0.75176275,  0.2578723 ],
        [-0.16892342,  0.47537887],
        [ 0.54853654, -0.73256385],
        [ 0.8891574 ,  0.21612257],
        [ 0.39133072,  0.13524306]],

       [[ 0.7525435 , -0.5251115 ],
        [ 0.81592697,  0.30876702],
        [ 0.3431359 ,  0.70552534],
        [ 0.19034834,  0.2761002 ],
        [ 0.5707368 , -0.48129925]]], dtype=float32)
Tensor("3_batches/

### Using LSTMCell (RNNCell 대신)

In [6]:
with tf.variable_scope('3_batches_dynamic_length', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
array([[[-0.08025328, -0.02257276],
        [-0.01457324,  0.00657582],
        [ 0.02442009, -0.04039814],
        [ 0.06362677, -0.07554006],
        [ 0.03316153,  0.112480

## Cost: sequence_loss
- logits : 예측
- targets : 정답

In [7]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])  # 정답

# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)
# -> one-hot: [1, 0, 1]

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loss:  0.5967595


In [8]:
# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
# -> one-hot: [1, 1, 1]
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)
# -> one-hot: [1, 1, 1] (하지만 정답에 더 가까움: loss작음)

prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
# -> one-hot: [0, 0, 0] 모두 오답
prediction4 = tf.constant([[[0, 1], [0, 1], [0, 1]]], dtype=tf.float32)
# -> one-hot: [1, 1, 1] 모두 정답

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction4, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval())
print("Loss2: ", sequence_loss2.eval())
print("Loss3: ", sequence_loss3.eval())
print("Loss4: ", sequence_loss4.eval())

Loss1:  0.5130153
Loss2:  0.3711007
Loss3:  1.3132616
Loss4:  0.31326166
